<a href="https://colab.research.google.com/github/PSamita/MIT-Course/blob/main/Hospital_LOS_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hospital Length of Stay (LOS) Prediction**


## **Context**

Hospital management is a vital area that has garnered significant attention during the COVID-19 pandemic. The inefficient distribution of resources, such as beds and ventilators, can potentially lead to numerous complications. However, this issue can be mitigated by accurately predicting the length of stay (LOS) for patients prior to admission. Once this information is determined, the hospital can effectively plan appropriate treatment, allocate resources, and arrange staff accordingly to reduce LOS and enhance the chances of recovery. Additionally, the allocation of rooms and beds can be optimized based on these predictions.

Due to its inefficient management system, HealthPlus hospital has experienced substantial financial losses and loss of life. They have encountered difficulties in equitably allocating equipment, beds, and hospital staff. Implementing a system capable of estimating the length of stay (LOS) for patients could significantly address this problem.

## **Objective**

The objective of this project is to analyze the data at HealthPlus as a Data Scientist, determine the key factors that significantly impact the length of stay (LOS), and create a machine learning model capable of predicting a patient's LOS using the data available during admission and after conducting relevant tests. Furthermore, the project aims to derive meaningful insights and develop policies from the data to assist the hospital in improving its healthcare infrastructure and increasing revenue.

## **Data Dictionary**

The data contains various information recorded during the time of admission of the patient. It only contains records of patients who were admitted to the hospital. The detailed data dictionary is given below:


* **patientid**: Patient ID
* **Age**: Range of age of the patient
* **gender**: Gender of the patient
* **Type of Admission**: Trauma, emergency or urgent
* **Severity of Illness**: Extreme, moderate, or minor
* **health_condition**s: Any previous health conditions suffered by the patient
* **Visitors with Patient**: The number of patients who accompany the patient
* **Insurance**: Does the patient have health insurance or not?
* **Admission_Deposit**: The deposit paid by the patient during admission
* **Stay (in days)**: The number of days that the patient has stayed in the hospital. This is the **target variable**
* **Available Extra Rooms in Hospital**: The number of rooms available during admission
* **Department**: The department which will be treating the patient
* **Ward_Facility_Code**: The code of the ward facility in which the patient will be admitted
* **doctor_name**: The doctor who will be treating the patient
* **staff_available**: The number of staff who are not occupied at the moment in the ward

## **Approach to solve the problem**

1. Import the necessary libraries.
2. Read the dataset and get an overview.
3. Conduct an exploratory data analysis.
4. Conduct data preprocessing.
5. Define the performance metric and build ML models
6. Check for assumptions
7. Compare models and determine the best one.
8. Report observations and business insights.

## **Importing Libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Removes the limit for the number of displayed columns
pd.set_option("display.max_columns", None)

# Sets the limit for the number of displayed rows
pd.set_option("display.max_rows", 200)

# To build models for prediction
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor

from sklearn.preprocessing import LabelEncoder

#for tuning the model
from sklearn.model_selection import GridSearchCV

# To check model performance
from sklearn.metrics import make_scorer,mean_squared_error, r2_score, mean_absolute_error

In [ ]:
#read the healthcare dataset file
data=pd.read_csv("/content/drive/MyDrive/MIT Data Science/healthcare_data.csv")

In [ ]:
# copying data to another variable to avoid any changes to original data
same_data = data.copy()

## **Data Overview**

In [ ]:
# View the first 5 rows of the dataset
data.head()

In [ ]:
# View the last 5 rows of the dataset
data.tail()

In [ ]:
#Understand the shape of the data
data.shape

- The dataset has **5,00,000 rows and 15 columns**

In [ ]:
#Checking the info of the data
data.info()

**Observations:**

-  Available Extra Rooms in Hospital , staff_available, patientid, Visitors with Patient, Admission_Deposit, and Stay (in days) are of **numeric data type** and the rest of the columns are of **object data type*.
- The number of non-null values is the same as the total number of entries in the data i.e. **there are no null values.**
- The column patientid is an identifier for patients in the data. 

In [ ]:
# checking for duplicate values in the Data
data.duplicated().sum()

**Observations:** 
- Data has unique rows only. There is no need to remove any rows.

In [ ]:
#To view patientid and the number of times they have visited the hospital
data['patientid'].value_counts()

**Observations:**
- **The maximum number of times the same patient admitted to the hospital is 21 and minimum is 1.**

In [ ]:
#Checking the descriptive statistics of the columns
data.describe().T

**Observations :**

* There are around **3 rooms available in the hospital on an average** and there are times when the hospital is full and there are no rooms available. The **maximum number of rooms available in the hospital are 24**.
* **On average there are around 5 staff personal available to treat the new patients** but it can also be zero at times. The maximum staff available in the hospital is 10.
* **On average around 3 visitors accompany the patient.** Some patients come on their own (minimum zero) and few cases have 32 visitors. It will be interesting to see if there is any relation between number of visitors and the severity of the patient.
* **The average admission deposit lies around 4722 dollars and a minimum of 1654 dollars is paid on every admission.**
* **Patient's stay has a large range from 3 to 51 days.** There might be outliers in this variable. The median length of stay is 9 days.


In [ ]:
# List of all important categorical variables
cat_col = ["Department", "Type of Admission", 'Severity of Illness', 'gender', 'Insurance', 'health_conditions', 'doctor_name', "Ward_Facility_Code", "Age"]

# Printing the number of occurrences of each unique value in each categorical column
for column in cat_col:
    print(data[column].value_counts(1))
    print("-" * 50)

**Observations :**

- **The majority of patients (~82%) admit to the hospital with moderate and minor illness** which is understandable as extreme illness is less frequent than moderate and minor illness. 
- **Gynecology department gets the most number of patients (~68%)** in the hospital whereas patients in Surgery department are very less (~1%).
- **Ward A and C accommodate the least number of patients (~12%).** These might be wards reserved for patient with extreme illness and patients who need surgery. It would be interesting to see if patients from these wards also stay for longer duration.
- **The majority of patients belong to the age group of 21-50 (\~75%) and are women (~75%).** The most number of patients in the gynecology department of the hospital can justify this.
- Most of the patients admitted to the hospital are the cases of trauma (~62%).
- **High Blood pressure and diabetes are the most common health conditions.**

## **Exploratory Data Analysis (EDA)**

### **Univariate Analysis**

In [ ]:
# function to plot a boxplot and a histogram along the same scale.


def histogram_boxplot(data, feature, figsize=(12, 7), kde=False, bins=None):
    """
    Boxplot and histogram combined

    data: dataframe
    feature: dataframe column
    figsize: size of figure (default (12,7))
    kde: whether to the show density curve (default False)
    bins: number of bins for histogram (default None)
    """
    f2, (ax_box2, ax_hist2) = plt.subplots(
        nrows=2,  # Number of rows of the subplot grid= 2
        sharex=True,  # x-axis will be shared among all subplots
        gridspec_kw={"height_ratios": (0.25, 0.75)},
        figsize=figsize,
    )  # creating the 2 subplots
    sns.boxplot(
        data=data, x=feature, ax=ax_box2, showmeans=True, color="violet"
    )  # boxplot will be created and a star will indicate the mean value of the column
    sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2, bins=bins, palette="winter"
    ) if bins else sns.histplot(
        data=data, x=feature, kde=kde, ax=ax_hist2
    )  # For histogram
    ax_hist2.axvline(
        data[feature].mean(), color="green", linestyle="--"
    )  # Add mean to the histogram
    ax_hist2.axvline(
        data[feature].median(), color="black", linestyle="-"
    )  # Add median to the histogram

#### **Length of stay**

In [ ]:
histogram_boxplot(data, "Stay (in days)", kde=True, bins=30)

**Observations:**

- There are **fewer number of patients staying more than 10 days in the hospital and very few who stay for more than 40 days**. This might be because the majority of patients are admitted for moderate or minor illness. 
- The peak of the distribution shows that the the most of the patients stay for 8-9 days in the hospital. 

#### **Admission deposit**

In [ ]:
histogram_boxplot(data, "Admission_Deposit", kde=True, bins=30)

**Observations:**

- The **distribution of the admission fees is close to normal with outliers on both sides**. There are few patients paying high amount of admission fees and few patients paying low amount of admission fees.

#### **Visitors with patients**

In [ ]:
histogram_boxplot(data, "Visitors with Patient", kde=True, bins=30)

**Observations:**

- The distribution of the number of visitors with the patient is **highly skewed towards right**.
- **2 and 4 are the most common number of visitors with patients.**

### **Bivariate Analysis**

In [ ]:
#finding the correlation between various columns of the dataset
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(15,7))
sns.heatmap(data.corr(), annot=True, vmin=-1, vmax=1, fmt=".2f",cmap="Spectral")

**Observations:** 
- The heatmap shows that there is **no correlation between variables**.
- The continuous variables shows no correlation with the target variable (Stay (in days)) which indicates that the **categorical variables might be more important for the prediction.**

In [ ]:
# function to plot stacked bar plots


def stacked_barplot(data, predictor, target):
    """
    Print the category counts and plot a stacked bar chart

    data: dataframe
    predictor: independent variable
    target: target variable
    """
    count = data[predictor].nunique()
    sorter = data[target].value_counts().index[-1]
    tab1 = pd.crosstab(data[predictor], data[target], margins=True).sort_values(
        by=sorter, ascending=False
    )
    print(tab1)
    print("-" * 120)
    tab = pd.crosstab(data[predictor], data[target], normalize="index").sort_values(
        by=sorter, ascending=False
    )
    tab.plot(kind="bar", stacked=True, figsize=(count + 1, 5))
    plt.legend(
        loc="lower left",
        frameon=False,
    )
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.show()

**Let's start by checking the distribution of the LOS for the various wards**

In [ ]:
sns.barplot(y = 'Ward_Facility_Code',x = 'Stay (in days)',data = data)
plt.show()

**Observations:**

- The hypothesis we made earlier is correct i.e. **wards A and C has the patients staying for the longest duration.**

In [ ]:
stacked_barplot(data, "Ward_Facility_Code", "Department")

**Observations:**

- **Ward Facility B, D and F are dedicated only to the gynecology department.**
- Wards A, C, and E have patients with all other diseases and patients undergoing surgery are admitted in ward A only

**Usually, the more severe the illness, the more the LOS, let's check the distribution of severe patients in various wards**

In [ ]:
stacked_barplot(data, "Ward_Facility_Code", "Severity of Illness")

**Observations :**

- Ward A has the most extreme case as well as the longest length of stay in the hospital. It might require more staff and resources as compared to other wards.
- Ward F has a large number of minor cases and Ward E has large number of moderate cases.

**Age can also be an important factor to find the length of stay. Let's check the same.**

In [ ]:
sns.barplot(y = 'Age',x = 'Stay (in days)',data = data )
plt.show()

**Observations:**

- **Patients aged between 1-10 and 51-100 tend to stay the most number of days in the hospital.** This might be because the majority of the patients between 21-50 age group gets admitted to the gynecology department and patients in age groups 1-10 and 5-100 might get admitted due to some other serious illness. 

## **Data Preparation for Model Building**

- Before we proceed to build a model, we'll have to encode categorical features.
- Separate the independent variables and dependent Variable.
- We'll split the data into train and test to be able to evaluate the model that we train on the training data.

In [ ]:
# Creating dummy variables for the categorical columns
#drop_first=True is used to avoid redundant variables
data = pd.get_dummies(
    data,
    columns=data.select_dtypes(include=["object", "category"]).columns.tolist(),
    drop_first=True,
)

In [ ]:
#Check the data, after handling categorical data
data

In [ ]:
#Dropping patientid from the data as it is an identifier and will not add value to the analysis
data=data.drop(columns=["patientid"])

In [ ]:
#Separating independent variables and the target variable
x=data.drop('Stay (in days)',axis=1)

y=data['Stay (in days)'] 

In [ ]:
#Splitting the dataset into train and test datasets
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,shuffle=True,random_state=1)

In [ ]:
#Checking the shape of the train and test data
print("Shape of Training set : ", x_train.shape)
print("Shape of test set : ", x_test.shape)


## **Model Building**

* We will be using different metrics functions defined in sklearn like RMSE, MAE, and 𝑅2 for regression models evaluation.
* We will define a function to calculate MAPE and adjusted 𝑅2.
* The mean absolute percentage error (MAPE) measures the accuracy of predictions as a percentage, and can be calculated as the average absolute percent error for each predicted value minus actual values divided by actual values. It works best if there are no extreme values in the data and none of the actual values are 0.

In [ ]:
# function to compute adjusted R-squared
def adj_r2_score(predictors, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = predictors.shape[0]
    k = predictors.shape[1]
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


# function to compute MAPE
def mape_score(targets, predictions):
    return np.mean(np.abs(targets - predictions) / targets) * 100


# function to compute different metrics to check performance of a regression model
def model_performance_regression(model, predictors, target):
    """
    Function to compute different metrics to check regression model performance

    model: regressor
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    r2 = r2_score(target, pred)  # to compute R-squared
    adjr2 = adj_r2_score(predictors, target, pred)  # to compute adjusted R-squared
    rmse = np.sqrt(mean_squared_error(target, pred))  # to compute RMSE
    mae = mean_absolute_error(target, pred)  # to compute MAE
    mape = mape_score(target, pred)  # to compute MAPE

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {
            "RMSE": rmse,
            "MAE": mae,
            "R-squared": r2,
            "Adj. R-squared": adjr2,
            "MAPE": mape,
        },
        index=[0],
    )

    return df_perf

In [ ]:
# RMSE
def rmse(predictions, targets):
    return np.sqrt(((targets - predictions) ** 2).mean())


# MAPE
def mape(predictions, targets):
    return np.mean(np.abs((targets - predictions)) / targets) * 100


# MAE
def mae(predictions, targets):
    return np.mean(np.abs((targets - predictions)))


# Model Performance on test and train data
def model_pref(olsmodel, x_train, x_test, y_train,y_test):

    # Insample Prediction
    y_pred_train = olsmodel.predict(x_train)
    y_observed_train = y_train

    # Prediction on test data
    y_pred_test = olsmodel.predict(x_test)
    y_observed_test = y_test

    print(
        pd.DataFrame(
            {
                "Data": ["Train", "Test"],
                "RMSE": [
                    rmse(y_pred_train, y_observed_train),
                    rmse(y_pred_test, y_observed_test),
                ],
                "MAE": [
                    mae(y_pred_train, y_observed_train),
                    mae(y_pred_test, y_observed_test),
                ],
                "MAPE": [
                    mape(y_pred_train, y_observed_train),
                    mape(y_pred_test, y_observed_test),
                ],
            }
        )
    )


In [ ]:
import statsmodels.api as sm

# Statsmodel api does not add a constant by default. We need to add it explicitly.
x_train1 = sm.add_constant(x_train)
# Add constant to test data
x_test1 = sm.add_constant(x_test)

# create the model
olsmodel1 = sm.OLS(y_train, x_train1).fit()

# get the model summary
olsmodel1.summary()
print(olsmodel1.summary())

In [ ]:
lin_reg_test = model_performance_regression(olsmodel1, x_test1, y_test)
lin_reg_test

- We can see that `R-squared` for the model is `0.84`. 
- Not all the variables are statistically significant enough to predict the outcome variable. To check which ones are statistically significant or have enough predictive power to predict the target variable, we need to check the `p-value` against all the independent variables.

**Interpreting the Regression Results:**

1. **Adjusted R-squared**: It reflects the fit of the model.
    - R-squared values range from 0 to 1, where a higher value generally indicates a better fit, assuming certain conditions are met.
    - In our case, the value for Adj. R-squared is **0.84**

2. **coef**: It represents the change in the output Y due to a change of one unit in the variable (everything else held constant).
3. **std err**: It reflects the level of accuracy of the coefficients.
    - The lower it is, the more accurate the coefficients are.
4. **P > |t|**: The p-value:
   
   * Pr(>|t|) : For each independent feature there is a null hypothesis and alternate hypothesis 

    **Ho:** Null Hypothesis - The independent feature is not significant 
   
    **Ha:** Alternate Hypothesis - The independent feature is significant 
    
   * A p-value of less than 0.05 is considered to be statistically significant.

   
5. **Confidence Interval**: It represents the range in which our coefficients are likely to fall (with a likelihood of 95%).



* Both the **R-squared and Adjusted R-squared of the model are around 84%**. This is a clear indication that we have been able to create a good model that is able to explain variance in the LOS of patients for up to 84%.

* We can examine the significance of the regression model, and try dropping insignificant variables.

In [ ]:
model_pref(olsmodel1, x_train1, x_test1,y_train,y_test)

**The Root Mean Squared Error** of train and test data is **not different**, indicating that **our model is not overfitting** to the training data.

Mean Absolute Error (MAE) indicates that our current model is able to predict LOS of patients within **mean error of 2.15 days** on the test data.

The units of both RMSE and MAE are same - days in this case. But RMSE is greater than MAE because it penalizes the outliers more.

**Mean Absolute Percentage Error is ~19%** on the test data.

### Checking for Multicollinearity 

We will use VIF, to check if there is multicollinearity in the data.

Features having a VIF score >5 will be dropped/treated till all the features have a VIF score <5

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor


def checking_vif(train):
    vif = pd.DataFrame()
    vif["feature"] = train.columns

    # calculating VIF for each feature
    vif["VIF"] = [
        variance_inflation_factor(train.values, i) for i in range(len(train.columns))
    ]
    return vif


In [ ]:
print(checking_vif(x_train1))

In [ ]:
# create the model
olsmodel1 = sm.OLS(y_train, x_train1).fit()

# get the model summary
olsmodel1.summary()

It is not a good practice to consider VIF values for dummy variables as they are correlated to other categories and hence have a high VIF usually. So we built the model and calculated the p-values. We can see that the categories in Insurance_Yes column shows p-value higher than 0.05. So we can drop that.

In [ ]:
x_train2 = x_train1.drop(['Insurance_Yes','staff_available','Visitors with Patient'] , axis=1)
print(checking_vif(x_train2))

## **Dropping the insignificant variables and creating the regression model again**

### **Examining the significance of the model**

It is not enough to just fit a multiple regression model to the data, it is also necessary to check whether all the regression coefficients are significant or not. Significance here means whether the population regression parameters are significantly different from zero.

From the above it may be noted that the regression coefficients corresponding to staff_available, Visitors with Patient, and Insurance_Yes **are not statistically significant at level α = 0.05.** In other words, the regression coefficients corresponding to these three are not significantly different from 0 in the population. 

Suppose you have a nominal categorical variable having 4 categories (or levels). You would create 3 dummy variables (k-1 = 4-1 dummy variables) and set one category as a reference level. Suppose one of them is insignificant.  Then if you exclude that dummy variable, it would change the reference level as you are indirectly combining that insignificant level with the original reference level. It would have a new reference level and interpretation would change. Moreover, excluding the level may make the others insignificant. If all the categories in a column shows p-value higher than 0.05 we can drop that.

**Hence, we will eliminate these three features and create a new model.**

In [ ]:
# create the model after dropping variables
x_train2 = x_train1.drop(['Insurance_Yes','staff_available','Visitors with Patient'] , axis=1)
x_test2 = x_test1.drop(['Insurance_Yes','staff_available','Visitors with Patient'] , axis=1)
# create the model
olsmodel2 = sm.OLS(y_train, x_train2).fit()
# get the model summary
olsmodel2.summary()

In [ ]:
lin_reg_test = model_performance_regression(olsmodel2, x_test2, y_test)
lin_reg_test

### **Checking the performance of the model on the train and test data set**

In [ ]:
model_pref(olsmodel2, x_train2, x_test2,y_train,y_test)

**Observations:**

* RMSE, MAE, and MAPE of train and test data are not very different, indicating that the **model is not overfitting and has generalized well.**

## **Checking for assumptions and rebuilding the model**

In this step, we will check for the below assumptions in the model, to check if they hold true or not. And if there is any issue, then we will rebuild the model after fixing those issues


1. Mean of residuals should be 0
2. No Heteroscedasticity
3. Linearity of variables
4. Normality of error terms

### **Mean of residuals should be 0 and normality of error terms**

In [ ]:
residual = olsmodel2.resid # Residuals

In [ ]:
residual.mean()

The mean of residuals is very close to 0. Hence, the corresponding assumption is satisfied.

## **Tests for Normality**

**What is the test?**

* Error terms/Residuals should be normally distributed

* If the error terms are non- normally distributed, confidence intervals may become too wide or narrow. Once confidence interval becomes unstable, it leads to difficulty in estimating coefficients based on minimization of least squares.

**What do non-normality indicate?**

* It suggests that there are a few unusual data points which must be studied closely to make a better model.

**How to check the normality?**

* It can be checked via QQ Plot. Residuals following normal distribution will make a straight line plot otherwise not.

* Other test to check for normality : Shapiro-Wilk test.

**What is the residuals are not-normal?**

* We can apply transformations like log, exponential, arcsinh, etc as per our data

In [ ]:
# Plot histogram of residuals
sns.histplot(residual, kde=True)

The residuals have a close to normal distribution. The assumption of normality is satisfied.

### **Linearity of Variables**

It states that the predictor variables must have a linear relation with the dependent variable.

To test the assumption, we'll plot residuals and fitted values on a plot and ensure that residuals do not form a strong pattern. They should be randomly and uniformly scattered on the x-axis.

In [ ]:
# predicted values
fitted = olsmodel2.fittedvalues

# sns.set_style("whitegrid")
sns.residplot(x = fitted, y = residual, color="lightblue")
plt.xlabel("Fitted Values")
plt.ylabel("Residual")
plt.title("Residual PLOT")
plt.show()

**Observations**
- We can see that there is no pattern in the residuals vs fitted values scatter plot now i.e. the linearity assumption is satisfied.

Let's check the final assumption

### **No Heteroscedasticity**

#### **Test for Homoscedasticity**

* **Homoscedasticity -** If the variance of the residuals are symmetrically distributed across the regression line, then the data is said to homoscedastic.

* **Heteroscedasticity -** If the variance is unequal for the residuals across the regression line, then the data is said to be heteroscedastic. In this case the residuals can form an arrow shape or any other non symmetrical shape.

We will use Goldfeld–Quandt test to check homoscedasticity.

Null hypothesis : Residuals are homoscedastic

Alternate hypothesis : Residuals are hetroscedastic

alpha = 0.05 

In [ ]:
import statsmodels.stats.api as sms
from statsmodels.compat import lzip

name = ["F statistic", "p-value"]
test = sms.het_goldfeldquandt(residual, x_train2)
lzip(name, test)

As we can see from the above test the p-value is greater than 0.05, so we fail to reject the null-hypothesis. That means - residuals are homoscedastic.

In [ ]:
coef = olsmodel2.params
coef

In [ ]:
# Let us write the equation of the fit
Equation = "Stay (in days)="
print(Equation, end='\t')
for i in range(len(coef)):
    print('(', coef[i], ') * ', coef.index[i], '+', end = ' ')

### **Interpreting our Regression Coefficients**

With our linear regression model's adjusted R-squared value of around 0.84, we are able to capture **84% of the variation** in our data.

The p-values for these variables are < 0.05 in our final model, meaning they are statistically significant towards Stay (in days) prediction.

* The Stay (in days) decreases with an increase in Department_radiotherapy. 1 unit increase in the Department_radiotherapy leads to a decrease of Stay (in days) ~ 4.62 times the Stay (in days) than the Department_TB&Chest_Disease that serves as a reference variable when everything else is constant.

* The Stay (in days) increases with an increase in Department_anesthesia. 1 unit increase in Department_anesthesia leads to an increase of Stay (in days) ~ 6.08 times the Stay (in days) than the Department_TB&Chest_Disease that serves as a reference variable when everything else is constant. This is understandable, as anesthesia is used in severe cases which results in more days of stay.

* The Stay (in days) increases with an increase in Department_surgery. 1 unit increase in Department_surgery leads to an increase of Stay (in days) ~ 9.68 times the Stay (in days) than the Department_TB&Chest_Disease that serves as a reference variable when everything else is constant. This is understandable, as surgery is conducted in severe cases which results in more days of stay.

* The Stay (in days) increases with an increase in doctor_name_Dr Simon. 1 unit increase in doctor_name_Dr Simon leads to an increase of Stay (in days) ~ 6.14 times the Stay (in days) than the doctor_name_Dr Isaac that serves as a reference variable when everything else is constant. This is understandable, as surgery cases are handled by Dr.Simon.


### **Applying the cross validation technique to improve the model and evaluating it using different evaluation metrics**

In [ ]:
# import the required function
from sklearn.model_selection import cross_val_score

# build the regression model using Sklearn Linear regression
linearregression = LinearRegression()                                    

cv_Score11 = cross_val_score(linearregression, x_train2, y_train, cv = 10) #cv=10 represents data is divided into 10 folds.
cv_Score12 = cross_val_score(linearregression, x_train2, y_train, cv = 10, 
                             scoring = 'neg_mean_squared_error')                                  


print("RSquared: %0.3f (+/- %0.3f)" % (cv_Score11.mean(), cv_Score11.std() * 2))
print("Mean Squared Error: %0.3f (+/- %0.3f)" % (-1*cv_Score12.mean(), cv_Score12.std() * 2))

**Observations:**
- The R-squared on the cross validation is 0.843, whereas on the training dataset it was 0.843
- And the MSE on cross validation is 9.831, whereas on the training dataset also it was 9.83

We may want to reiterate the model building process again with new features or better feature engineering to increase the R-squared and decrease the MSE on cross validation.

### **Ridge Regression**

In [ ]:
rdg = Ridge()

In [ ]:
rdg.fit(x_train, y_train)
model_pref(rdg, x_train, x_test,y_train,y_test)

In [ ]:
ridge_regression_perf_test = model_performance_regression(rdg, x_test, y_test)

ridge_regression_perf_test

Ridge regression is able to produce similar results in comparison to Linear Regression.

There is definitely some scope to improve the model's performance, as there is a feeling that we are not able to fully model the relationship using linear models.

Let's now build **Non-Linear Regression models** like Decision Tree Regressors and Random Forest Regressors and check their performance.

### **Decision Tree Regressor**

In [ ]:
#Decision Tree Regressor
dt_regressor = DecisionTreeRegressor(random_state = 1)

#Fitting the model
dt_regressor.fit(x_train,y_train)

# Model Performance on test data i.e prediction
dt_regressor_perf_test = model_performance_regression(dt_regressor, x_test, y_test)

dt_regressor_perf_test

### **Tuning the Decision Tree Regressor**

In [ ]:
# Choose the type of regressor 
dtree_tuned = DecisionTreeRegressor(random_state=1)

# Grid of parameters to choose from
parameters = {'max_depth': np.arange(2,8), 
              'criterion': ['squared_error', 'friedman_mse'],
              'min_samples_leaf': [1, 3, 5, 7],
              'max_leaf_nodes' : [2, 5, 7] + [None]
             }


# Type of scoring used to compare parameter combinations
scorer = make_scorer(r2_score)

# Run the grid search
grid_obj = GridSearchCV(dtree_tuned, parameters, scoring=scorer,cv=5)
grid_obj = grid_obj.fit(x_train,y_train)

# Set the dtree_tuned_regressor to the best combination of parameters
dtree_tuned_regressor = grid_obj.best_estimator_

dtree_tuned_regressor.fit(x_train, y_train)

**We have tuned the model and fit the tuned model on the training data. Now, let's check the model performance on the testing data**

In [ ]:
#Get the score of tuned decision tree regressor
dtree_tuned_regressor_perf_test = model_performance_regression(dtree_tuned_regressor, x_test, y_test)

In [ ]:
dtree_tuned_regressor_perf_test

**Let's look at the feature importance of the tuned decision tree model**

In [ ]:
#Plotting the feature importance
features = list(x.columns)
importances = dtree_tuned_regressor.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(10,10))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='violet', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

**Observations:**

- **Department_gynecology, Age_41_50, and Age_31_40 are the most important features** followed by Department_anesthesia, Department_anesthesia, Department_surgery.
- The rest of the variables have no impact in this model to decide the duration of the stay in Hospital.

### **Bagging Regressor**

In [ ]:
#Bagging Regressor
bagging_estimator=BaggingRegressor(random_state=1)

#Fitting the model
bagging_estimator.fit(x_train,y_train)

# Model Performance on test data i.e prediction
bagging_estimator_perf_test = model_performance_regression(bagging_estimator, x_test, y_test)

bagging_estimator_perf_test

### **Tuned Bagging Regressor** 

In [ ]:
# Choose the type of regressor. 
bagging_tuned = BaggingRegressor(random_state=1)

# Grid of parameters to choose from
parameters = {"n_estimators": [10,15,20],
              "max_samples" : [0.8,1],
              "max_features" : [0.8,1]
             }


# Type of scoring used to compare parameter combinations
scorer = make_scorer(r2_score)

# Run the grid search
grid_obj = GridSearchCV(bagging_tuned , parameters, scoring=scorer,cv=5)
grid_obj = grid_obj.fit(x_train,y_train)

# Set the bagging_tuned_regressor to the best combination of parameters
bagging_tuned_regressor = grid_obj.best_estimator_

bagging_tuned_regressor.fit(x_train, y_train)

In [ ]:
#Get the score of tuned Bagging tree regressor
bagging_tuned_regressor_perf_test = model_performance_regression(bagging_tuned_regressor, x_test, y_test)
bagging_tuned_regressor_perf_test

**The bagging regressor has no attribute to calculate feature importance in Sklearn. So, let's move on to the next model** 

### **Random Forest Regressor**

In [ ]:
#Random Forest Regressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 1)

#Fitting the model
regressor.fit(x_train, y_train)

# Model Performance on test data i.e prediction
regressor_perf_test = model_performance_regression(regressor, x_test, y_test)

regressor_perf_test

### **Tuned Random Forest Regressor**

**NOTE:** Due to the large number of observations in the data, the code in the next cell for GridSearchCV might take 1-2 hours to run depending on the configuration of your system.  

In [ ]:
rf_tuned = RandomForestRegressor(random_state=1)

# Grid of parameters to choose from
parameters = {"n_estimators": [110, 120],
    "max_depth": [5, 7],
    "max_features": [0.8, 1]
             }

# Type of scoring used to compare parameter combinations
scorer = make_scorer(r2_score)

# Run the grid search
grid_obj = GridSearchCV(rf_tuned, parameters, scoring=scorer,cv=5)
grid_obj = grid_obj.fit(x_train,y_train)

# Set the rf_tuned_regressor to the best combination of parameters
rf_tuned_regressor = grid_obj.best_estimator_

rf_tuned_regressor.fit(x_train, y_train)

In [ ]:
# Model Performance on test data i.e prediction
rf_tuned_regressor_perf_test = model_performance_regression(rf_tuned_regressor, x_test, y_test)

rf_tuned_regressor_perf_test

**Observations:**

- The performance of the tuned model is actually decreased as compared to the model with default parameters. 
- This might be because we have tried very less number of hyperparameters and less number of values due to computational limits. We can certainly try to improve the performance of the model by tuning the model further.   

**Visualizing the feature importance**

In [ ]:
#Plotting the feature importance
importances = rf_tuned_regressor.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(10,10))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='violet', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

**Observations:**

- The top 3 important variables are the same as decision tree model. There are some other features like gender, and doctors adding some value to the model.

**Model Performance Comparison**

In [ ]:
models_test_comp_df = pd.concat(
    [
        lin_reg_test.T,
        ridge_regression_perf_test.T,
        dt_regressor_perf_test.T,
        regressor_perf_test.T,
        bagging_estimator_perf_test.T,
        dtree_tuned_regressor_perf_test.T,
        bagging_tuned_regressor_perf_test.T,
        rf_tuned_regressor_perf_test.T
    ],
    axis=1,
)

models_test_comp_df.columns = [
    "Linear Regression",
    "Ridge Regression",
    "Decision tree regressor",
    "Random Forest regressor",
    "Bagging regressor",
    "Tuned Decision Tree regressor",
    "Tuned Bagging Tree regressor",
    "Tuned Random Forest Regressor"]

print("Test performance comparison:")
models_test_comp_df

**Observations:**

- All the other models are giving a good performance in terms of RMSE and R-squared when compared to linear and ridge regression.
- The bagging and random forest models are performing better than a single decision tree.
- The random forest model with default parameters is giving the best performance among all the trained models.

### **Fitting the chosen final model**

- Final Model Building - Random Forest Regressor
- We will consider Random Forest Regressor with default parameters as our final model.

In [ ]:
final_model = RandomForestRegressor(n_estimators = 100, random_state = 1)
final_model.fit(x_train, y_train)

In [ ]:
final_model_perf_train = model_performance_regression(final_model, x_train, y_train)

In [ ]:
final_model_perf_train

* **Both the R-squared and the Adjusted R-squared of the model are approx 99.6% on the training data.** This indicates that the model is able to explain approx full variance in the target variable using the independent variables.

* Let's do a quick performance check on the test data.

In [ ]:
final_model_test_perf = model_performance_regression(final_model, x_test, y_test)
print("Test Performance:")
final_model_test_perf

* The model is giving good performance on the test data as well i.e. the model is giving generalized performance.
* The units of both RMSE and MAE are the same, days in this case. But RMSE is greater than MAE because it penalizes the outliers more.
* **The MAE < 1 indicates that the model is able to predict the length of stay within a mean error of 1 day,** which is a very good performance.
* **MAPE of 7.31 on the test data indicates that the model can predict within ~7% of the actual length of stay of patients.**

## **Observations**



1.   Random Forest Regressor model gives the best results. It is successful in capturing ~98% of variations in the test data.
2.   MAE of 0.75 indicates that the model can succesfully predict the length of stay of a patient during admission with just an error of 1 day. 
3. Factors like visitors with **patient** and **admission deposit** plays an important role in the prediction.
4. Factors like **staff_available** and **extra rooms** has very less to do with the predition from the model.



## **Business Insights and Recommendations**

- Gynecology is the busiest department of the hospital and it handles 68.7% of the total number of patients. It needs ample resources and staff for the smooth functioning of the department.
- The number of visitors with the patients highly influences the length of stay of a patient. The maximum number of visitors can go up to 32 which is very high. A restriction can be imposed on this.
- 74.2% of the patients are female. Thus, resources need to be procured while keeping this figure in mind.
- A large number of patients (89.3%) of the patients are in trauma or emergency during admission. An increase in ambulances and emergency rooms can reduce the risk of casualties.
- Ward A has the most number of patients who stay for the longest and the most serious patients. These wards can be equipped with more resources and staff to reduce the length of stay of these patients.
- Elderly patients (51-100) and children (1-10) stay for the longest. Extra attention to these age groups can lead to a faster discharge from the hospital.
- Wards D, E, and C have the most visitors with a patient. These wards will need more space and amenities like washrooms, shops, and lobbies for the visitors. Spaces can also be rented out to shop owners and advertisements to generate extra income.
- Finally, the Random Forest Regressor can predict the length of stay of the patient with just an error of 1 day. The hospital can use these predictions to allocate the resources and staff accordingly and reduce any kind of wastage. The hospital can also allocate the wards and doctors accordingly to optimize admissions even during emergencies.

## **Additional Content (Optional)**

## **Boosting Models**

Let's now look at the other kind of Ensemble technique - Boosting Models

### **XGBoost**
- XGBoost stands for Extreme Gradient Boosting.
- XGBoost is a tree-based ensemble machine learning technique that improves prediction power and performance by improvising on the Gradient Boosting framework and incorporating some reliable approximation algorithms. It is widely utilized and routinely appears at the top of competition leader boards in data science.


In [ ]:
#installing the xgboost library using "pip' command.
!pip install xgboost

In [ ]:
#importing the Random Forest Regressor and Bagging Regressor [Bagging]
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor

#importing the AdaBoostRegressor and GradientBoostingRegressor [Boosting]
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

#importing the XGBReressor from the xgboost
from xgboost import XGBRegressor

In [ ]:
#Adaboost Regressor
adaboost_model = AdaBoostRegressor(random_state=1)

#Fitting the model
adaboost_model.fit(x_train,y_train)

# model Performance on test data i.e prediction
adaboost_model_perf_test = model_performance_regression(adaboost_model, x_test, y_test)

adaboost_model_perf_test

In [ ]:
#Gradient Boost Regressor
gbc = GradientBoostingRegressor(random_state=1)
gbc.fit(x_train,y_train)
gbc_perf_test = model_performance_regression(gbc, x_test, y_test)

In [ ]:
gbc_perf_test

In [ ]:
#XGBoost Regressor
xgb = XGBRegressor(random_state=1, eval_metric='logloss')
xgb.fit(x_train,y_train)
xgb_perf_test = model_performance_regression(xgb, x_test, y_test)


In [ ]:
xgb_perf_test

### **Hyperparameter Tuning: Boosting**

Hyperparameter tuning is a great technique in machine learning to develop the model with optimal parameters. If the size of the data increases, the computation time will increase during training process.
- For the practice purposes, we have listed below some of the important hyperparameters for each algorithm that can be tuned to improve the model performance.

**1. Adaboost**

- Some important hyperparameters that can be tuned:
  - **base_estimator** object, default=None
The base estimator from which the boosted ensemble is built. If None, then the base estimator is DecisionTreeRegressor initialized with max_depth=3.
  - **n_estimators** int, default=50
  The maximum number of estimators at which boosting is terminated. In the case of a perfect fit, the learning procedure is stopped early.
  - **loss :** {‘linear’, ‘square’, ‘exponential’}, default=’linear’
The loss function to use when updating the weights after each boosting iteration.
 - **learning_rate** float, default=1.0
Weight applied to each regressor at each boosting iteration. A higher learning rate increases the contribution of each regressor.

**2. Gradient Boosting Algorithm**

- - Some important hyperparameters that can be tuned:
   - **n_estimators**: the number of boosting stages that will be performed.
 - **max_depth**: limits the number of nodes in the tree. The best value depends on the interaction of the input variables.

  - **min_samples_split**: the minimum number of samples required to split an internal node.

  - **learning_rate**: how much the contribution of each tree will shrink.

  - **loss**: loss function to optimize. 

For a better understanding of each parameter in Gradient Boosting Regressor, please refer to this [source](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor).



**3. XGBoost Algorithm**

- - Some important hyperparameters that can be tuned:
  - **booster** [default= gbtree ] Which booster to use. Can be gbtree, gblinear or dart; gbtree and dart use tree-based models while gblinear uses linear functions.
  - **min_child_weight** [default=1]

    The minimum sum of instance weight (hessian) needed in a child. If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight, then the building process will give up further partitioning. In the linear regression task, this simply corresponds to the minimum number of instances needed to be in each node. The larger min_child_weight is, the more conservative the algorithm will be.

  For a better understanding of each parameter in XGBoost Regressor, please refer to this [source](https://xgboost.readthedocs.io/en/stable/parameter.html).

### **Comparison of all the models we have built so far**

In [ ]:
models_test_comp_df = pd.concat(
    [
        lin_reg_test.T,
        ridge_regression_perf_test.T,
        dt_regressor_perf_test.T,
        regressor_perf_test.T,
        bagging_estimator_perf_test.T,
        adaboost_model_perf_test.T,
        gbc_perf_test.T,
        xgb_perf_test.T

    ],
    axis=1,
)

models_test_comp_df.columns = [
    "Linear Regression",
    "Ridge Regression",
    "Decision tree regressor",
    "Random Forest regressor",
    "Bagging regressor",
    "Adaboost regressor",
    "Gradientboost regressor",
    "XGBoost regressor"
]

print("Test performance comparison:")

In [ ]:
models_test_comp_df

**Observations :**

* With default parameters, the bagging methods outperform the boosting methods.
* **The Random Forest Regressor gives the best performance for this dataset.**